In [1]:
import sys, os
sys.path.append(os.path.abspath('../src/'))

In [2]:
import utils
import pandas as pd

In [3]:
train, _ = utils.read_current('../data/round_254/')

In [4]:
train['era'] = train.era.str.extract('(\d+|X)$', expand=False).str.zfill(4)

In [5]:
features = [f for f in train.columns if f.startswith('feature')]
len(features)

310

In [6]:
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.metrics import make_scorer

from scipy.stats import spearmanr

In [7]:
from lightgbm import LGBMRegressor

In [12]:
param = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "l2",
    "num_iterations": 2000,
    "learning_rate": 0.006,
    "lambda_l1": 1.4,
    "lambda_l2": 1.0,
    "bagging_fraction": 0.55,
    "bagging_freq": 1,
    "num_leaves": 107,
    "max_depth": 15,
    "verbose": 0,
    "random_state": 0,
}

In [13]:
def spearman(y_true, y_pred, axis=0):
    """ Calculate Spearman correlation """
    return spearmanr(y_true, y_pred, axis=axis)[0]

In [14]:
cv = GroupKFold(n_splits=10)

model = LGBMRegressor(**param)

X = train[features]
y = train.target
groups = train.era

In [ ]:
n_scores = cross_val_score(model, X, y, groups=groups, scoring=make_scorer(spearman), cv=cv)

In [16]:
n_scores

array([0.03858683, 0.03508479, 0.04694068, 0.04788679, 0.0376488 ,
       0.05353759, 0.04606174, 0.05328889, 0.04667806, 0.04705327])

In [18]:
model.fit(train[features], train.target)

[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.085530 seconds.
You can set `force_row_wi

LGBMRegressor(bagging_fraction=0.55, bagging_freq=1, lambda_l1=1.4,
              lambda_l2=1.0, learning_rate=0.006, max_depth=15, metric='l2',
              num_iterations=2000, num_leaves=107, objective='regression',
              random_state=0, verbose=0)

In [19]:
model.booster_.save_model('../models/lgb001.txt')